In [4]:
import pandas as pd
from datetime import datetime
from datetime import date
import numpy as np

In [5]:
df = pd.read_parquet('../do_not_commit/Datasets/Persist_PC_BIOS_DATA.pq')


# bios_release_date_age


In [6]:
# Number of days since the BIOS version was released.
# Calculate this from the 'ReleaseDate00'.
# Take the .max() 'ReleaseDate00' to account for duplicate records for machineID 1681622.
bios_release_date = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['ReleaseDate00'].agg('max').reset_index().rename(columns={'ReleaseDate00':'BIOS Release Date'})
bios_release_date

,MachineID,RWB_EFFECTIVE_DATE,BIOS Release Date
0,16783564,2023-01-19,2018-12-12 00:00:00.000
1,16783564,2023-01-20,2018-12-12 00:00:00.000
2,16783564,2023-01-21,2018-12-12 00:00:00.000
3,16783564,2023-01-22,2018-12-12 00:00:00.000
4,16783564,2023-01-23,2018-12-12 00:00:00.000
...,...,...,...
1042055,16818954,2023-06-28,2022-08-18 00:00:00.000
1042056,16818955,2023-06-28,2023-02-24 00:00:00.000
1042057,16818958,2023-06-28,2022-08-18 00:00:00.000
1042058,16818959,2023-06-28,2022-07-06 00:00:00.000


In [7]:
def calculate_age_in_days(target_date_str, in_date):

    # Convert target_date_str to a Python datetime object
    target_date = datetime.strptime(target_date_str, '%Y-%m-%d').date()

    # Calculate the age in days
    age_in_days = (in_date - target_date).days

    return age_in_days


def create_bios_features(in_df):

    # Change effective date to type date
    in_df['BIOS Release Date'] = pd.to_datetime(in_df['BIOS Release Date']).dt.strftime('%Y-%m-%d')

    # Get age in days
    todaydate = date(2023, 7, 4)
    in_df['BIOSReleaseAge'] = in_df['BIOS Release Date'].apply(lambda x: calculate_age_in_days(x, todaydate)
    if pd.notnull(x) else np.nan)
    return in_df

In [8]:
df = create_bios_features(bios_release_date)

In [9]:
df

,MachineID,RWB_EFFECTIVE_DATE,BIOS Release Date,BIOSReleaseAge
0,16783564,2023-01-19,2018-12-12,1665.0
1,16783564,2023-01-20,2018-12-12,1665.0
2,16783564,2023-01-21,2018-12-12,1665.0
3,16783564,2023-01-22,2018-12-12,1665.0
4,16783564,2023-01-23,2018-12-12,1665.0
...,...,...,...,...
1042055,16818954,2023-06-28,2022-08-18,320.0
1042056,16818955,2023-06-28,2023-02-24,130.0
1042057,16818958,2023-06-28,2022-08-18,320.0
1042058,16818959,2023-06-28,2022-07-06,363.0


In [10]:
df.to_parquet('../do_not_commit/FeatureDatasets/pc_bios_data.pq')